In [6]:
import pandas as pd
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split

# Загрузка данных
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Конвертация времени из строки в datetime-объект pandas
train['ts'] = pd.to_datetime(train['ts'])
test['ts'] = pd.to_datetime(test['ts'])

# Выделение временных признаков
for df in [train, test]:
    df['hour'] = df['ts'].dt.hour
    df['day'] = df['ts'].dt.dayofweek
    df['day_of_month'] = df['ts'].dt.day
    df['month'] = df['ts'].dt.month

# Категориальные признаки для модели CatBoost
categorical_features = ['gate_id', 'hour', 'day', 'day_of_month', 'month']

# Определяем целевую переменную и признаки
X = train[categorical_features]
y = train['user_id']
X_test = test[categorical_features]

# Создание Pool-объектов для CatBoost
train_pool = Pool(X, y, cat_features=categorical_features)
test_pool = Pool(X_test, cat_features=categorical_features)

# Параметры модели CatBoost (могут быть настроены для улучшения результатов)
model = CatBoostClassifier(iterations=1000, 
                           learning_rate=0.1, 
                           depth=10,
                           loss_function='MultiClass',
                           verbose=True,
                           task_type="GPU",
                           devices='0:1')

# Обучение модели
model.fit(train_pool)

# Получение предсказаний на тестовых данных
test_pred_proba = model.predict_proba(test_pool)  # Вероятности для каждого класса
test_pred = model.predict(test_pool).flatten()    # Предсказанные классы

# Преобразование предсказанных вероятностей в уникальные предсказания
test['user_id_proba'] = test_pred_proba.max(axis=1)  # Макс. вероятность для каждого примера
test['predicted_user_id'] = test_pred  # Предсказанный user_id

# Определение наиболее вероятного user_id для каждого user_word
output = test.groupby('user_word').apply(lambda x: x.loc[x['user_id_proba'].idxmax()])

# Создание окончательной таблицы submission
submit = output[['user_word', 'predicted_user_id']].rename(columns={'predicted_user_id': 'preds'})
submit.to_csv('submission.csv', index=False)

print("Прогнозы сохранены в файл submission.csv")

0:	learn: 3.9099474	total: 205ms	remaining: 3m 24s
1:	learn: 3.8236909	total: 408ms	remaining: 3m 23s
2:	learn: 3.7201722	total: 614ms	remaining: 3m 24s
3:	learn: 3.6454802	total: 814ms	remaining: 3m 22s
4:	learn: 3.5849259	total: 1.01s	remaining: 3m 22s
5:	learn: 3.5359461	total: 1.22s	remaining: 3m 21s
6:	learn: 3.4954364	total: 1.42s	remaining: 3m 21s
7:	learn: 3.4615099	total: 1.62s	remaining: 3m 21s
8:	learn: 3.4061238	total: 1.83s	remaining: 3m 21s
9:	learn: 3.3589635	total: 2.03s	remaining: 3m 21s
10:	learn: 3.3210961	total: 2.24s	remaining: 3m 21s
11:	learn: 3.2878173	total: 2.45s	remaining: 3m 21s
12:	learn: 3.2573190	total: 2.65s	remaining: 3m 21s
13:	learn: 3.2262995	total: 2.87s	remaining: 3m 21s
14:	learn: 3.1980441	total: 3.08s	remaining: 3m 22s
15:	learn: 3.1731202	total: 3.29s	remaining: 3m 22s
16:	learn: 3.1535667	total: 3.51s	remaining: 3m 22s
17:	learn: 3.1351672	total: 3.72s	remaining: 3m 23s
18:	learn: 3.1166705	total: 3.94s	remaining: 3m 23s
19:	learn: 3.0909039	t

In [1]:
import pandas as pd
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split

# Загрузка данных
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Конвертация времени из строки в datetime-объект pandas
train['ts'] = pd.to_datetime(train['ts'])
test['ts'] = pd.to_datetime(test['ts'])

# Выделение временных признаков
for df in [train, test]:
    df['hour'] = df['ts'].dt.hour
    df['day'] = df['ts'].dt.dayofweek
    df['day_of_month'] = df['ts'].dt.day
    df['month'] = df['ts'].dt.month

# Категориальные признаки для модели CatBoost
categorical_features = ['gate_id', 'hour', 'day', 'day_of_month', 'month']

# Определяем целевую переменную и признаки
X = train[categorical_features]
y = train['user_id']
X_test = test[categorical_features]

# Создание Pool-объектов для CatBoost
train_pool = Pool(X, y, cat_features=categorical_features)
test_pool = Pool(X_test, cat_features=categorical_features)

# Параметры модели CatBoost (могут быть настроены для улучшения результатов)
model = CatBoostClassifier(iterations=1000, 
                           learning_rate=0.1, 
                           depth=3,
                           loss_function='MultiClass',
                           verbose=True,
                           task_type="GPU",
                           devices='0:1')

# Обучение модели
model.fit(train_pool)

# Получение предсказаний на тестовых данных
test_pred_proba = model.predict_proba(test_pool)  # Вероятности для каждого класса
test_pred = model.predict(test_pool).flatten()    # Предсказанные классы

# Создаем DataFrame для работы с предсказаниями
pred_df = pd.DataFrame({
    'user_word': test['user_word'],
    'preds': test_pred,
    'proba': test_pred_proba.max(axis=1) # Максимальная вероятность для каждого прогноза
})

# Сортируем предсказания по убыванию уверенности (вероятности)
pred_df_sorted = pred_df.sort_values(by='proba', ascending=False)

# Отбираем уникальные предсказания
used_ids = set()
unique_predictions = []

for index, row in pred_df_sorted.iterrows():
    if row['preds'] not in used_ids:
        unique_predictions.append(row)
        used_ids.add(row['preds'])

# Создаем финальный DataFrame из уникальных предсказаний
final_sub_df = pd.DataFrame(unique_predictions)

# Теперь переупорядочим final_sub_df согласно исходному порядку user_word в test
final_submission = test[['user_word']].merge(final_sub_df, on='user_word', how='left')

# Замена пропущенных значений (если какие-то user_word не получили предсказания)
final_submission['preds'].fillna(-999, inplace=True) # Для новых пользователей

# Сохранение результатов в файл CSV
final_submission[['user_word', 'preds']].to_csv('submission.csv', index=False)

print("Уникальные прогнозы сохранены в файл submission.csv.")


0:	learn: 3.9138355	total: 17.8ms	remaining: 17.8s
1:	learn: 3.8301773	total: 32.8ms	remaining: 16.4s
2:	learn: 3.7320819	total: 47.3ms	remaining: 15.7s
3:	learn: 3.6666683	total: 61.6ms	remaining: 15.3s
4:	learn: 3.6155712	total: 75ms	remaining: 14.9s
5:	learn: 3.5703954	total: 88.2ms	remaining: 14.6s
6:	learn: 3.5349690	total: 102ms	remaining: 14.4s
7:	learn: 3.5046465	total: 115ms	remaining: 14.3s
8:	learn: 3.4658472	total: 129ms	remaining: 14.2s
9:	learn: 3.4374277	total: 141ms	remaining: 14s
10:	learn: 3.4117355	total: 154ms	remaining: 13.9s
11:	learn: 3.3911641	total: 168ms	remaining: 13.8s
12:	learn: 3.3723344	total: 182ms	remaining: 13.8s
13:	learn: 3.3517278	total: 196ms	remaining: 13.8s
14:	learn: 3.3352439	total: 210ms	remaining: 13.8s
15:	learn: 3.3095475	total: 223ms	remaining: 13.7s
16:	learn: 3.2931580	total: 237ms	remaining: 13.7s
17:	learn: 3.2768540	total: 252ms	remaining: 13.8s
18:	learn: 3.2607040	total: 265ms	remaining: 13.7s
19:	learn: 3.2470866	total: 279ms	remai

In [6]:
import pandas as pd
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split

# Загрузка данных
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Конвертация времени из строки в datetime-объект pandas
train['ts'] = pd.to_datetime(train['ts'])
test['ts'] = pd.to_datetime(test['ts'])

# Выделение временных признаков
for df in [train, test]:
    df['hour'] = df['ts'].dt.hour
    df['day'] = df['ts'].dt.dayofweek
    df['day_of_month'] = df['ts'].dt.day
    df['month'] = df['ts'].dt.month

# Категориальные признаки для модели CatBoost
categorical_features = ['gate_id', 'hour', 'day', 'day_of_month', 'month']

# Определяем целевую переменную и признаки
X = train[categorical_features]
y = train['user_id']
X_test = test[categorical_features]

# Создание Pool-объектов для CatBoost
train_pool = Pool(X, y, cat_features=categorical_features)
test_pool = Pool(X_test, cat_features=categorical_features)

# Параметры модели CatBoost (могут быть настроены для улучшения результатов)
model = CatBoostClassifier(iterations=1000, 
                           learning_rate=0.1, 
                           depth=10,
                           loss_function='MultiClass',
                           verbose=True,
                           task_type="GPU",
                           devices='0:1')

# Обучение модели
model.fit(train_pool)

# Получение предсказаний на тестовых данных
test_pred_proba = model.predict_proba(test_pool)  # Вероятности для каждого класса
test_pred = model.predict(test_pool).flatten()    # Предсказанные классы

# Создаем исходный DataFrame с предсказаниями и соответствующими вероятностями
predictions_df = pd.DataFrame({
    'user_word': test['user_word'],
    'predicted_id': test_pred, 
    'max_proba': test_pred_proba.max(axis=1)
})

# Чтобы убедиться, что мы не имеем повторяющихся user_word, мы можем сгруппировать данные и выбрать запись с максимальной вероятностью
unique_user_word_df = predictions_df.groupby('user_word').apply(lambda x: x.nlargest(1, 'max_proba')).reset_index(drop=True)

# Далее, чтобы убедиться, что каждый ID уникален, мы используем аналогичную логику
# Сортируем DataFrame по убыванию max_proba
unique_user_word_df.sort_values('max_proba', ascending=False, inplace=True)

# Затем удаляем дубликаты ID, сохраняя только первые вхождения (самые высокие вероятности)
final_df = unique_user_word_df.drop_duplicates(subset='predicted_id', keep='first')

# Если мы потеряли какие-то user_word после удаления дубликатов, нужно обработать это
expected_user_words = set(test['user_word'].unique())
missing_user_words = expected_user_words - set(final_df['user_word'])

# Для недостающих user_word установим предсказание в -999
missing_df = pd.DataFrame({'user_word': list(missing_user_words), 'predicted_id': [-999] * len(missing_user_words)})

# Объединяем исходный датафрейм с пропущенными значениями
final_df = pd.concat([final_df, missing_df], axis=0)

# Экспорт результатов в файл 'submission.csv'
final_df[['user_word', 'predicted_id']].to_csv('submission.csv', index=False)

print("Файл submission.csv с уникальными предсказаниями сохранён.")

0:	learn: 3.9099478	total: 202ms	remaining: 3m 21s
1:	learn: 3.8236905	total: 412ms	remaining: 3m 25s
2:	learn: 3.7201722	total: 620ms	remaining: 3m 25s
3:	learn: 3.6454798	total: 824ms	remaining: 3m 25s
4:	learn: 3.5849263	total: 1.03s	remaining: 3m 24s
5:	learn: 3.5359461	total: 1.23s	remaining: 3m 23s
6:	learn: 3.4954364	total: 1.43s	remaining: 3m 23s
7:	learn: 3.4615095	total: 1.64s	remaining: 3m 22s
8:	learn: 3.4061234	total: 1.85s	remaining: 3m 23s
9:	learn: 3.3589639	total: 2.08s	remaining: 3m 26s
10:	learn: 3.3210959	total: 2.3s	remaining: 3m 26s
11:	learn: 3.2878173	total: 2.5s	remaining: 3m 26s
12:	learn: 3.2573190	total: 2.71s	remaining: 3m 26s
13:	learn: 3.2262993	total: 2.94s	remaining: 3m 27s
14:	learn: 3.1980439	total: 3.16s	remaining: 3m 27s
15:	learn: 3.1731202	total: 3.39s	remaining: 3m 28s
16:	learn: 3.1535661	total: 3.61s	remaining: 3m 28s
17:	learn: 3.1351678	total: 3.82s	remaining: 3m 28s
18:	learn: 3.1166709	total: 4.04s	remaining: 3m 28s
19:	learn: 3.0909034	tot

In [5]:
import pandas as pd
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split

# Загрузка данных
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Конвертация времени из строки в datetime-объект pandas
train['ts'] = pd.to_datetime(train['ts'])
test['ts'] = pd.to_datetime(test['ts'])

# Выделение временных признаков
for df in [train, test]:
    df['hour'] = df['ts'].dt.hour
    df['day'] = df['ts'].dt.dayofweek
    df['day_of_month'] = df['ts'].dt.day
    df['month'] = df['ts'].dt.month

# Категориальные признаки для модели CatBoost
categorical_features = ['gate_id', 'hour', 'day', 'day_of_month', 'month']

# Определяем целевую переменную и признаки
X = train[categorical_features]
y = train['user_id']
X_test = test[categorical_features]

# Создание Pool-объектов для CatBoost
train_pool = Pool(X, y, cat_features=categorical_features)
test_pool = Pool(X_test, cat_features=categorical_features)

# Параметры модели CatBoost (могут быть настроены для улучшения результатов)
model = CatBoostClassifier(iterations=500, 
                           learning_rate=0.1, 
                           depth=8,
                           loss_function='MultiClass',
                           verbose=True,
                           task_type="GPU",
                           devices='0:1')

# Обучение модели
model.fit(train_pool)

# Получение предсказаний на тестовых данных
test_pred_proba = model.predict_proba(test_pool)  # Вероятности для каждого класса
test_pred = model.predict(test_pool).flatten()    # Предсказанные классы

# Создаем исходный DataFrame с предсказаниями и соответствующими вероятностями
predictions_df = pd.DataFrame({
    'user_word': test['user_word'],
    'predicted_id': test_pred, 
    'max_proba': test_pred_proba.max(axis=1)
})
# Сортируем по вероятности убыванию для каждого user_word
predictions_df = predictions_df.sort_values(by='max_proba', ascending=False)

# Шаг 1: Удаляем дубликаты по 'user_word', оставляя только записи с самыми высокими вероятностями
deduped_by_user_word = predictions_df.drop_duplicates(subset='user_word', keep='first')

# Шаг 2: Удаляем дубликаты по 'predicted_id', оставляя только записи с самыми высокими вероятностями
deduped_by_id = deduped_by_user_word.drop_duplicates(subset='predicted_id', keep='first')

# Проверяем наличие каждого 'user_word' из тестового набора в дедуплицированном наборе
missing_user_words = set(test['user_word']) - set(deduped_by_id['user_word'])

# Шаг 3: Для недостающих 'user_word' устанавливаем предсказания в -999
missing_entries = pd.DataFrame({
    'user_word': list(missing_user_words),
    'predicted_id': [-999] * len(missing_user_words),
    'max_proba': [0] * len(missing_user_words)  # Нулевые вероятности для заполнения
})

# Объединяем дедуплицированные записи с недостающими записями
final_submission = pd.concat([deduped_by_id, missing_entries])

# Упорядочим строки в исходном порядке test['user_word'] для соответствия порядку сабмита
final_ordered_submission = test[['user_word']].merge(final_submission, on='user_word', how='left')

# Сохраняем предсказания в итоговый CSV файл
final_ordered_submission[['user_word', 'predicted_id']].to_csv('submission.csv', index=False)

print("Готово: Уникальные прогнозы сохранены в файл submission.csv.")

0:	learn: 3.9099482	total: 63.9ms	remaining: 31.9s
1:	learn: 3.8236913	total: 126ms	remaining: 31.3s
2:	learn: 3.7164390	total: 190ms	remaining: 31.4s
3:	learn: 3.6425666	total: 254ms	remaining: 31.5s
4:	learn: 3.5851612	total: 319ms	remaining: 31.6s
5:	learn: 3.5375062	total: 381ms	remaining: 31.4s
6:	learn: 3.4972013	total: 444ms	remaining: 31.3s
7:	learn: 3.4639721	total: 507ms	remaining: 31.2s
8:	learn: 3.4116293	total: 571ms	remaining: 31.2s
9:	learn: 3.3677864	total: 638ms	remaining: 31.3s
10:	learn: 3.3342187	total: 702ms	remaining: 31.2s
11:	learn: 3.3036138	total: 767ms	remaining: 31.2s
12:	learn: 3.2762008	total: 831ms	remaining: 31.1s
13:	learn: 3.2544039	total: 896ms	remaining: 31.1s
14:	learn: 3.2318889	total: 961ms	remaining: 31.1s
15:	learn: 3.2078927	total: 1.03s	remaining: 31.1s
16:	learn: 3.1871085	total: 1.1s	remaining: 31.2s
17:	learn: 3.1679521	total: 1.17s	remaining: 31.2s
18:	learn: 3.1469547	total: 1.24s	remaining: 31.4s
19:	learn: 3.1288598	total: 1.32s	remaini